# Tractography (TCK, TRK, TRX, VTK)

### Installation
This notebook can be run on a local server or on a virtualized environement like Google Colab.


##### Current limitations:
This notebook runs through `jupyter notebook` or `jupyter lab` but gets unresponsive when using JupyterLite (e.g., notebooks ran through VS Code or Google Colab).

In [1]:
#run if using Google Colab or not building from source
try:
    # -q silence the output from pip. Remove this flag if you need 
    # to debug an installation issue.
    %pip install -q ipyniivue
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
from ipyniivue import Niivue
import ipywidgets as widgets
from time import sleep, time
from jupyter_ui_poll import ui_events

### Display Controls
interactive controls to customize visualization

In [3]:
# Fiber Length Slider
length_slider = widgets.IntSlider(min=1, max=80, value=3, description="Length")
def on_length_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberLength", change["new"])
length_slider.observe(on_length_change, names="value")

# Fiber Dither Slider
dither_slider = widgets.FloatSlider(min=0, max=10, value=1, description="Dither")
def on_dither_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberDither", change["new"] * 0.1)
dither_slider.observe(on_dither_change, names="value")

# Fiber Color Dropdown
color_dropdown = widgets.Dropdown(
    options=[
        ("Global direction", "Global"),
        ("Local direction", "Local"),
        ("Fixed", "Fixed"),
        ("First Per Vertex Type (if available)", "DPV0"),
        ("First Per Streamline Type (if available)", "DPS0"),
    ],
    value="Global",
    description="Fiber color:",
)

def on_color_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberColor", change["new"])
color_dropdown.observe(on_color_change, names="value")

# Fiber Decimation Dropdown
decimation_dropdown = widgets.Dropdown(
    options=[
        ("100%", 1),
        ("50%", 2),
        ("25%", 4),
        ("10%", 10),
    ],
    value=1,
    description="Fiber reduction:",
)

def on_decimation_change(change):
    if change["name"] == "value":
        nv.set_mesh_property(nv.meshes[0].id, "fiberDecimationStride", change["new"])
decimation_dropdown.observe(on_decimation_change, names="value")

# Arrange controls vertically
controls = widgets.VBox([length_slider, dither_slider, color_dropdown, decimation_dropdown])

### Niivue

In [4]:
# Creating a NiiVue object
nv = Niivue(back_color=[0.8, 0.8, 1, 1], show_3D_crosshair=True)

# Display it so it has a canvas an WebGL context. Also display the controls
display(widgets.VBox([controls, nv]))

### Load a volume and set slice type

In [5]:
nv.add_volume("https://niivue.github.io/niivue/images/mni152.nii.gz")
nv.set_slice_type(nv.slice_type.render)

### Load a mesh

In [6]:
nv.load_meshes([{"url": "https://niivue.github.io/niivue/images/dpsv.trx", "rgba255": [0, 142, 0, 255]}])

The following is in a separate cell because we have to wait for the typescript end of the widget to update .meshes variable in the python end. 

In [7]:
################
# This block of code is currently necessary to avoid nv.meshes[0] being accessed before the
# mesh becomes accessible, in which case the notebook crashes with an IndexError.
# This error is triggered for example when the used hit "cell -> Run all"
################
timeout = 60
start = time()
i = 1
with ui_events() as poll:
    while True:
        poll(1)
        if len(nv.meshes):
            break
        if time() > start + timeout:
            raise TimeoutError
################
            
nv.set_mesh_property(nv.meshes[0].id, "colormap", "blue")
nv.set_clip_plane([-0.1, 270, 0])